# DT Assignment1

# Data Loading

In [ ]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')



In [ ]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [ ]:
def get_gini(df, label):
    p_y=len(df[df[label]=='yes'])/len(df[label])  #타켓변수의 값이 yes인 것의 확률
    p_n = 1- p_y  #타켓변수의 값이 no인 것의 확률
    
    gini=1-p_y**2-p_n**2 #각각의 확률을 제곱해서 1에서 빼주면 gini 계수가 나옴
    return gini    

In [ ]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [ ]:
from itertools import combinations

def get_binary_split(df, attribute):
  attri_class=df[attribute].unique() #feature에 있는 class를 리스트로 받음
  result=[] #이진분류를 담을 리스트

  for i in range (1,len(attri_class)): #for문을 class의 갯수만큼 반복

    for j in combinations(attri_class,i): #1개, 2개 ~~ class갯수 -1개의 class 조합을 구함

      result+=[list(j)] 그 조합을 이진분류를 담는 리스트에 추가
  return result

In [ ]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [ ]:

def get_attribute_gini_index(df, attribute, label):
  total_split=get_binary_split(df,attribute) #total_split에 해당 attribute의 이진분류 할당
  result={} #feature에 class당 gini index를 저장

  for key in total_split: #for문을 돌면서 이진분류를 하나씩 할당해 gini index를 구함

    true_df=df[df[attribute].isin(key)] #atrribute에 할당된 이진분류에 속한 class가 있는 df를 할당
    false_df=df[~df[attribute].isin(key)] #atrribute에 할당된 이진분류에 속하지 않은 class가 있는 df를 할당
    gini=(len(true_df)/len(df)*(get_gini(true_df,label)))+(len(false_df) /len(df)*(get_gini(false_df,label)))
    #할당된 이진분류에 속한 것의 gini index와 전체에서 비율을 곱함 + 할당된 이진분류에 속하지 않은 것의 gini index와 전체에서 비율을 곱함

    result['_'.join(key)]=gini
    #해당 값과 할당되었던 이진분류를 딕셔너리 형태로 저장

  return result


In [ ]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'middle_aged': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937,
 'senior': 0.45714285714285713,
 'youth': 0.3936507936507937,
 'youth_middle_aged': 0.45714285714285713,
 'youth_senior': 0.35714285714285715}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [ ]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [ ]:
##문제1 답안
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [ ]:


def get_feature(df, target):
    gini_dict={} #attribution의 가장 작은 gini index를 구해서 딕셔너리 저장
    variables = df.columns[df.columns != target]
    results = [] #모든 attribution에 대한 gini index 담음
    for var in variables: #타켓변수 제외한 모든 attribution에 대하여 
        gini_var = get_attribute_gini_index(df, var, target) #해당 attribution에 이진분류에 따른 각각의 gini index 저장
        
        if gini_var!="": #해당 attribution의 이진분류에 대한 gini index가 존재하는 경우
          
          gini_dict[min(gini_var.keys())]=min(gini_var.values()) #해당 attribution에 대한 gini index 중 가장 작은 값을 골라 할당함
          results.append([gini_dict[min(gini_var.keys())],min(gini_var.keys()),var]) #여기에도 attribution, 해당 이진분류값, gini index를 리스트로 저장
    results.sort() #정렬
    return min(results) #가장 작은 값 반환


In [ ]:
get_feature(pd_data,"class_buys_computer")

[0.35714285714285715, 'middle_aged', 'age']

In [ ]:
##문제3 답안

def split_gini(pd,target):
  split=get_feature(pd,target) #한번 binary split 실시한 결과 할당받음

  split_gini,split_class,split_var=split[0],split[1],split[2] #gini index값,해당 이진분류,attribution까지 차례로 할당

  true_df=pd_data[pd_data[split_var]==split_class] #attribution에 그 이진분류(class)값들이 있는 df
  false_df=pd_data[pd_data[split_var]!=split_class]#attribution에 그 이진분류(class)값들이 없는 df
  true_df.drop(split_var,axis=1,inplace=True) #원할한 gini index 계산을 위해 그 attribution column은 삭제
  false_df.drop(split_var,axis=1,inplace=True) #


  print(split_class,":",get_feature(true_df,target))
  print("not",split_class,":",get_feature(false_df,target))
  

In [ ]:
split_gini(pd_data,"class_buys_computer")

middle_aged : [0.0, 'excellent', 'credit_rating']
not middle_aged : [0.31999999999999984, 'no', 'student']
